In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:


# Define paths to your dataset
train_path = '/content/drive/MyDrive/Riya/train4/'
validation_path = '/content/drive/MyDrive/Riya/validation4'


In [ ]:
# Data augmentation and rescaling for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Rescaling for validation data
validation_datagen = ImageDataGenerator(rescale=1./255)

# Loading training data
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(256, 256),
    batch_size=4,
    class_mode='binary'
)

# Loading validation data
validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(256, 256),
    batch_size=4,
    class_mode='binary'
)


Found 4601 images belonging to 2 classes.
Found 662 images belonging to 2 classes.


In [ ]:
# Load the VGG16 model, excluding the top (output) layers
base_model = tf.keras.applications.VGG16(include_top=False, input_shape=(256, 256, 3))

# Add custom layers on top of the pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

# Create the new model
model = Model(inputs=base_model.input, outputs=x)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

# Save the model
model.save('/content/drive/MyDrive/Riya/morphing_detection_model.h5')

Epoch 1/10
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 3931s 3s/step - accuracy: 0.9803 - loss: 0.0706 - val_accuracy: 1.0000 - val_loss: 3.7828e-04
Epoch 2/10
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 1.0000 - loss: 1.1864e-06 - val_accuracy: 1.0000 - val_loss: 7.7157e-05
Epoch 3/10
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 3911s 3s/step - accuracy: 0.9983 - loss: 0.0042 - val_accuracy: 1.0000 - val_loss: 5.7547e-05
Epoch 4/10
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 1.0000 - loss: 3.4815e-07 - val_accuracy: 1.0000 - val_loss: 4.2510e-06
Epoch 5/10
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 3835s 3s/step - accuracy: 0.9994 - loss: 0.0021 - val_accuracy: 1.0000 - val_loss: 3.1046e-05
Epoch 6/10
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 32s 25ms/step - accuracy: 1.0000 - loss: 3.8334e-06 - val_accuracy: 1.0000 - val_loss: 1.1706e-04
Epoch 7/10
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 3869s 3s/step - accuracy: 0.9999 - loss: 5.5922e-04 - val_accuracy: 1.0000 - val_loss: 8.1608e-05
Epoch 8/10
1150/1150 ━━━━━━━━━━━━━━━

In [ ]:
# Load the model for inference
loaded_model = load_model('/content/drive/MyDrive/Riya/morphing_detection_model.h5')

In [ ]:




# Example of making predictions
# Assuming 'image_path' is the path to your test image
# Upload the test image from dataset to the'check' folder to verify the results

#image_path = '/content//drive/MyDrive/Riya/check/127_034.jpg'
#image_path = '/content//drive/MyDrive/Riya/check/137_043.jpg'
#image_path = '/content//drive/MyDrive/Riya/check/136_08.jpg'
#image_path = '/content//drive/MyDrive/Riya/check/132_03.jpg'
#image_path = '/content//drive/MyDrive/Riya/check/128_138.jpg'
#image_path = '/content//drive/MyDrive/Riya/check/094_100.jpg'
#image_path = '/content//drive/MyDrive/Riya/check/135_08.jpg'
#image_path = '/content//drive/MyDrive/Riya/check/132_141.jpg'
#image_path = '/content//drive/MyDrive/Riya/check/134_144.jpg'


img = tf.keras.preprocessing.image.load_img(image_path, target_size=(256, 256))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
predictions = loaded_model.predict(img_array)
print('working')
print(predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 903ms/step
working
[[1.]]


In [ ]:
# Threshold to decide the class
threshold = 0.5

# Values based on observed outputs
morphed_output = 1.4080911812610198e-31
original_output = 2.6642618957997334e-30

# Calculate the custom threshold
custom_threshold = (morphed_output + original_output) / 2
print(f'custom threshold: {custom_threshold}')
# Get the prediction value
prediction_value = predictions[0][0]

# Determine the predicted class based on the threshold
predicted_class = 'morphed' if prediction_value <= threshold else 'original'

print(f'Prediction value: {prediction_value}')
print(f'Predicted class: {predicted_class}')


custom threshold: 1.4025355069629177e-30
Prediction value: 1.0
Predicted class: original
